In [1]:
!pip install -U -q langchain openai ragas arxiv pymupdf chromadb wandb tiktoken unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import getpass
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


# Download SEC filing

In [2]:
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import UnstructuredURLLoader

url = "https://www.sec.gov/Archives/edgar/data/0001559720/000155972023000020/abnb-20230930.htm"
loader = UnstructuredURLLoader(urls=[url], headers={'User-Agent': 'jay jay@jay.com'})
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Generate a test dataset

In [4]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

# generator with openai models
generator = TestsetGenerator.with_openai()



In [5]:
TestsetGenerator.generate()

TypeError: TestsetGenerator.generate() missing 2 required positional arguments: 'self' and 'test_size'

In [6]:
# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/60 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
# visualize the dataset as a pandas DataFrame
dataframe = testset.to_pandas()
dataframe.head(10)

,question,contexts,ground_truth,evolution_type,episode_done
0,What is the reason for the increase in payroll...,"[, or 14%, for the three months ended Septembe...",The reason for the increase in payroll-related...,simple,True
1,What types of investments are included in the ...,[ 529 530 % Interest expense $ (5) $ (6) (20)...,The types of investments included in the inves...,simple,True
2,What is the purpose of the 2023 Share Repurcha...,[32\n\nTable of Contents\n\nMaterial Cash Requ...,The purpose of the 2023 Share Repurchase Progr...,simple,True
3,How is Free Cash Flow calculated and why is it...,[ such taxes; and\n\nAdjusted EBITDA does not ...,Free Cash Flow is calculated as net cash provi...,simple,True
4,What criteria must be met for a company to be ...,[owned subsidiaries and variable interest enti...,The criteria that must be met for a company to...,simple,True
5,What was the percentage change in general and ...,"[, or 14%, for the three months ended Septembe...",General and administrative expense increased b...,reasoning,True
6,Why is revenue presented on a constant currenc...,[ current period revenue from the correspondin...,The presentation of revenue on a constant curr...,reasoning,True
7,What was the net income for Q3 2023 compared t...,[ — Income before income taxes 44 50 25 33 Pro...,nan,multi_context,True
8,What caused the Company's net income to increa...,"[ necessary, appealing the Tax Court’s decisio...",nan,multi_context,True
9,What was the change in provision for (benefit ...,[ 529 530 % Interest expense $ (5) $ (6) (20)...,The provision for (benefit from) income taxes ...,simple,True


In [9]:
dataframe.shape

(10, 5)